## 保存和恢复模型

In [1]:
import sys
import os

import tensorflow as tf
from tensorflow import keras


print("python version: ", sys.version_info)
for module in tf, keras:
    print(module.__name__, "version:",module.__version__)

python version:  sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
tensorflow version: 2.3.0
tensorflow.keras version: 2.4.0


### 1. 配置

#### 1.1 安装

```shell
pip install -q pyyaml h5py
```

#### 1.2 获取示例数据集

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### 1.3 定义模型

In [3]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer="adam", 
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=["accuracy"])
    
    return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 2. 训练期间保存模型 checkpoints

 **keras.callbacks.ModelCheckpoint** 允许在训练的过程中和结束时回调保存的模型。

#### 2.1 Checkpoint 回调用法

In [4]:
checkpoint_path = "./checkpoints/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

model.fit(train_images, 
          train_labels, 
          epochs=10, 
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
23/32 [====================>.........] - ETA: 0s - loss: 1.3318 - accuracy: 0.6114
Epoch 00001: saving model to ./checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 1.1688 - accuracy: 0.6650 - val_loss: 0.7050 - val_accuracy: 0.7960
Epoch 2/10
21/32 [==================>...........] - ETA: 0s - loss: 0.4709 - accuracy: 0.8646
Epoch 00002: saving model to ./checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4207 - accuracy: 0.8840 - val_loss: 0.5399 - val_accuracy: 0.8300
Epoch 3/10
21/32 [==================>...........] - ETA: 0s - loss: 0.2749 - accuracy: 0.9286
Epoch 00003: saving model to ./checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2991 - accuracy: 0.9160 - val_loss: 0.4732 - val_accuracy: 0.8540
Epoch 4/10
20/32 [=================>............] - ETA: 0s - loss: 0.2171 - accuracy: 0.9438
Epoch 00004: saving model to ./checkpoints/trainin

这将创建一个 TensorFlow checkpoint 文件集合，这些文件在每个 epoch 结束时更新：

In [5]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


创建一个新的未经训练的模型。仅恢复模型的权重时，必须具有与原始模型具有相同网络结构的模型。由于模型具有相同的结构，您可以共享权重，尽管它是模型的不同实例。

In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3080 - accuracy: 0.0950
Untrained model, accuracy:  9.50%


然后从 checkpoint 加载权重并重新评估：

In [7]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4081 - accuracy: 0.8700
Untrained model, accuracy: 87.00%


#### 2.2 checkpoint回调选项

回调提供了几个选项，为 checkpoint 提供唯一名称并调整 checkpoint 频率。  
  
训练一个新模型，每五个 epochs 保存一次唯一命名的 checkpoint ：  

In [8]:
checkpoint_path = "./checkpoints/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5,  # period已被废除，用save_freq替换
    #     save_freq=5
)

model = create_model()
# "checkpoint_path"格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0,
)


Epoch 00005: saving model to ./checkpoints/training_2/cp-0005.ckpt

Epoch 00010: saving model to ./checkpoints/training_2/cp-0010.ckpt

Epoch 00015: saving model to ./checkpoints/training_2/cp-0015.ckpt

Epoch 00020: saving model to ./checkpoints/training_2/cp-0020.ckpt

Epoch 00025: saving model to ./checkpoints/training_2/cp-0025.ckpt

Epoch 00030: saving model to ./checkpoints/training_2/cp-0030.ckpt

Epoch 00035: saving model to ./checkpoints/training_2/cp-0035.ckpt

Epoch 00040: saving model to ./checkpoints/training_2/cp-0040.ckpt

Epoch 00045: saving model to ./checkpoints/training_2/cp-0045.ckpt

Epoch 00050: saving model to ./checkpoints/training_2/cp-0050.ckpt


In [9]:
!ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'./checkpoints/training_2/cp-0050.ckpt'

注意: 默认的 tensorflow 格式仅保存最近的5个 checkpoint 。  
  
如果要进行测试，请重置模型并加载最新的 checkpoint ：  

In [11]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,test_labels, verbose=1)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 [==============================] - 0s 1ms/step - loss: 0.4833 - accuracy: 0.8770
Restored model, accuracy: 87.70%


### 3. 这些文件是什么？

注意: 默认的 tensorflow 格式仅保存最近的5个 checkpoint 。  
  
如果要进行测试，请重置模型并加载最新的 checkpoint ：  
* 一个或多个包含模型权重的分片。
* 索引文件，指示哪些权重存储在哪个分片中。  
  
如果你只在一台机器上训练一个模型，你将有一个带有后缀的碎片： .data-00000-of-00001  

### 4. 手动保存权重

您将了解如何将权重加载到模型中。使用 Model.save_weights 方法手动保存它们同样简单。默认情况下， tf.keras (特别是save_weights ） 使用 TensorFlow checkpoints 格式 .ckpt 扩展名和 ( 保存在 HDF5 扩展名为 .h5 保存并序列化模型 )：

In [12]:
model.save_weights("./checkpoints/my_checkpoint")
model = create_model()
model.load_weights("./checkpoints/my_checkpoint")

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4833 - accuracy: 0.8770
Restored model, accuracy: 87.70%


### 5. 保存整个模型

调用 model.save 将保存模型的结构，权重和训练配置保存在单个文件/文件夹中。这可以让您导出模型，以便在不访问原始 Python 代码\*的情况下使用它。因为优化器状态（optimizer-state）已经恢复，您可以从中断的位置恢复训练。  
  
整个模型可以以两种不同的文件格式（SavedModel 和 HDF5）进行保存。需要注意的是 TensorFlow 的 SavedModel 格式是 TF2.x. 中的默认文件格式。但是，模型仍可以以 HDF5 格式保存。下面介绍了以两种文件格式保存整个模型的更多详细信息。  
  
保存完整模型会非常有用——您可以在 TensorFlow.js（Saved Model, HDF5）加载它们，然后在 web 浏览器中训练和运行它们，或者使用 TensorFlow Lite 将它们转换为在移动设备上运行（Saved Model, HDF5）  
  
\*自定义对象（例如，子类化模型或层）在保存和加载时需要特别注意。请参阅下面的保存自定义对象部分  

#### 5.1 SavedModel 格式

In [13]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save("saved_model/my_model")

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1797 - accuracy: 0.6690
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4410 - accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2951 - accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2206 - accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1549 - accuracy: 0.9670
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [14]:
!ls saved_model

my_model


In [15]:
!ls saved_model/my_model

assets         saved_model.pb variables


从保存的模型重新加载新的 Keras 模型:

In [16]:
new_model = keras.models.load_model("saved_model/my_model")

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


还原的模型使用与原始模型相同的参数进行编译。 尝试使用加载的模型运行评估和预测：

In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4135 - accuracy: 0.0810
Restored model, accuracy:  8.10%
(1000, 10)


#### 5.2 HDF5 格式

Keras使用 HDF5 标准提供了一种基本的保存格式。

In [18]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save("my_model.h5")

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1444 - accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4297 - accuracy: 0.8740
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3000 - accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2267 - accuracy: 0.9360
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1714 - accuracy: 0.9580


现在，从该文件重新创建模型：

In [19]:
new_model = keras.models.load_model("my_model.h5")
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


检查其准确率（accuracy）：

In [20]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4535 - accuracy: 0.0870
Restored model, accuracy:  8.70%


Keras 通过检查网络结构来保存模型。这项技术可以保存一切:  
* 权重值  
* 模型的架构  
* 模型的训练配置(您传递给编译的内容)  
* 优化器及其状态（如果有的话）（这使您可以在中断的地方重新开始训练）

Keras 无法保存 v1.x 优化器（来自 tf.compat.v1.train），因为它们与checkpoint不兼容。对于 v1.x 优化器，您需要在加载-失去优化器的状态后，重新编译模型。

#### 5.3 保存自定义对象

如果使用的是 SavedModel 格式，则可以跳过此部分。  
HDF5 和 SavedModel 之间的主要区别在于，HDF5 使用对象配置保存模型结构，而 SavedModel 保存执行图。  
因此，SavedModel 能够保存自定义对象，例如子类化模型和自定义层，而无需原始代码。  

要将自定义对象保存到 HDF5，必须执行以下操作:  
1. 在对象中定义一个 get_config 方法，以及可选的 from_config 类方法。
    * get_config(self) 返回重新创建对象所需的参数的 JSON 可序列化字典。
    * from_config(cls, config) 使用从 get_config 返回的 config 来创建一个新对象。默认情况下，此函数将使用 config 作为初始化 kwargs（return cls(\*\*config)）。
2. 加载模型时，将对象传递给 custom_objects 参数。参数必须是将字符串类名称映射到 Python 类的字典。例如，tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})  
